In [9]:
import pandas as pd
import requests

# Load the CSV file
word_dictionary = pd.read_csv("../data/processed/word_dictionary.csv", sep=',')

api_key = ""

def get_definition(word, api_key):
    url = f"https://stdict.korean.go.kr/api/search.do?certkey_no=6879&key={api_key}&type_search=search&req_type=json&q={word}"
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        items = data['channel'].get('item', [])
        if items:
            # Get the first definition from the first sense of the first item
            first_definition = items[0].get('sense', {}).get('definition', '')
            return first_definition
    return None

word_dictionary['definition'] = None

i=0
for idx, word in enumerate(word_dictionary['Word']):
    definition = get_definition(word, api_key)
    if definition:
        word_dictionary.at[idx, 'definition'] = definition
        i += 1
    if i % 50 == 0:
        print(f"{i} words processed")
        
# word_dictionary.to_csv("../data/processed/word_dictionary_with_definitions.csv", index=False)

50 words processed
100 words processed
150 words processed
200 words processed
250 words processed
300 words processed
350 words processed
400 words processed
450 words processed
500 words processed
550 words processed
600 words processed
650 words processed
700 words processed
750 words processed
800 words processed
850 words processed
900 words processed
950 words processed
1000 words processed
1050 words processed
1100 words processed
1150 words processed
1200 words processed
1250 words processed
1300 words processed
1350 words processed
1400 words processed
1450 words processed
1500 words processed
1550 words processed
1600 words processed
1650 words processed
1700 words processed
1750 words processed
1800 words processed
1850 words processed


In [10]:
word_dictionary = word_dictionary[word_dictionary['definition'].apply(lambda x: len(str(x)) > 15)].reset_index(drop=True)
word_dictionary.to_csv("../data/processed/word_dictionary_with_definitions.csv", index=False, encoding='utf-8-sig')

In [19]:
import openai
import pandas as pd

word_dictionary = pd.read_csv("../data/processed/word_dictionary_with_definitions.csv", sep=',', encoding='cp949')
OPEN_API_KEY = ''

openai.api_key = OPEN_API_KEY

def get_completion(definition, word, level):
    try:
        response = openai.chat.completions.create(
            messages=[
                {"role": "system", "content": "[역할] 당신은 초등학교 국어 선생님입니다. 해당 단어의 사전적 정보를 확인한 후 4지선다 문제를 출제하세요."},
                {"role": "user", "content": f"[방법]"
                                            f" 아래 주어진 지문과 정해진 질문 내용을 바탕으로 {level} 수준의 4지선다 형태의 객관식 문제 1개를 출제하세요. 단어의 사전적 정보를 확인하고 사전적정보는 그대로 출력하며 반드시 아래 Output의 유형에 맞는 구성으로 만드세요. 요구한 결과물만 출력해줘"
                                            f"[Input] "
                                            f"[단어]"
                                            f"{word}"
                                            f"[사전적 정보]  "
                                            f"{definition} "
                                            f"[난이도] - {level}"
                                            f""
                                            f"[Output]"
                                            f"{example}"}
            ],
            model="gpt-4o-mini"
        )
        # print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

example = """
{
    "question": "다음 정보를 보고 의미하는 단어를 고르세요",
    "definition" : "", 
    "options":  {
        "1":"보기A",
        "2":"보기b",
        "3":"보기c",
        "4":"보기d"
    },
    "answer": 2,
    "reason": "답에 대한 이유"
}
"""

responses = []
level_mapping = {
    1: "미취학아동",
    2: "초등학교 1학년",
    3: "초등학교 2학년",
    4: "초등학교 3학년",
    5: "초등학교 4학년",
    6: "초등학교 5학년",
    7: "초등학교 6학년"
}

for i, row in word_dictionary.iterrows():
    if (i+1)%50==0:
        print(i+1)
    education_level = level_mapping.get(row['Level'], "Unknown")
    response = get_completion(row['definition'], row['Word'], education_level)
    
    if response:
        # print(response)
        responses.append(response)
    else:
        responses.append("Failed to get a response from the API.")
        
# print(responses)

50
100
150
200
250
300
350
400
450
500
550
600
650
700
750
800
850
900
950
1000
1050
1100


In [20]:
responses_df = pd.DataFrame(word_dictionary[['Level']])
responses_df['response'] = responses

responses_df.to_csv("../data/processed/dt01_11.csv", index=False, encoding='utf-8-sig')

In [21]:
import json
from datetime import datetime

def transform_row(row):
    response_json = json.loads(row['response'])
    options = response_json.get('options', {})
    
    answer_key = str(response_json.get("answer", ""))  
    answer_value = ""
    for k, v in options.items():
        if str(k) == answer_key:
            answer_value = v  
            break

    now = datetime.now().isoformat()
    
    transformed = {
        "title": response_json.get("question", ""),
        "content": response_json.get("definition", ""),
        "answer" : answer_value,
        "solution": response_json.get("reason", ""),
        "questionTypeCode": "QT01",
        "problemLevelCode": f"PL0{row['Level']}",
        "problemTypeCode": "PT01",
        "problemTypeDetailCode": "PT0111",
        "metadata": {
            "options": [v for v in options.values()] 
        },
        "createdDate": now,
        "modifiedDate": now 
    }
    
    return transformed

transformed_data = []
for i, row in responses_df.iterrows():
    try:
        json.loads(row['response'])  # Try parsing the original JSON
        transformed_data.append(transform_row(row))
    except json.JSONDecodeError as e:
        print(f"Error in row {i}: {e}")
        print(f"Problematic JSON: {row['response']}")

    
dt01_11_df = pd.DataFrame(transformed_data)

Error in row 71: Expecting property name enclosed in double quotes: line 9 column 5 (char 189)
Problematic JSON: {
    "question": "다음 정보를 보고 의미하는 단어를 고르세요",
    "definition" : "비교가 되는 두 대상이 서로 같지 아니하다.",
    "options":  {
        "1":"같다",
        "2":"다르다",
        "3":"비슷하다",
        "4":"같은",
    },
    "answer": 2,
    "reason": "다르는 두 대상을 비교했을 때 서로 같지 않다는 뜻이에요."
}
Error in row 693: Expecting property name enclosed in double quotes: line 9 column 5 (char 184)
Problematic JSON: {
    "question": "다음 정보를 보고 의미하는 단어를 고르세요",
    "definition" : "넘칠 정도로 매우 가득하거나 수북한 모양.",
    "options":  {
        "1":"가득",
        "2":"듬뿍",
        "3":"무리",
        "4":"빈",
    },
    "answer": 2,
    "reason": "듬뿍은 넘칠 정도로 가득 차 있는 상태를 나타내는 단어입니다."
}
Error in row 779: Expecting property name enclosed in double quotes: line 9 column 5 (char 181)
Problematic JSON: {
    "question": "다음 정보를 보고 의미하는 단어를 고르세요",
    "definition": "사물이 전하여 내려온 그 처음.",
    "options": {
        "1": "새로움",
        "2": "원래",
  

In [22]:
print(dt01_11_df)
with open("../data/processed/problem_data/pt0111.json", 'w', encoding='utf-8') as f:
    json.dump(transformed_data, f, indent=4, ensure_ascii=False)
dt01_11_df.to_csv("../data/processed/problem_data/pt0111.csv", index=False, encoding='utf-8-sig')

                        title  \
0     다음 정보를 보고 의미하는 단어를 고르세요   
1     다음 정보를 보고 의미하는 단어를 고르세요   
2     다음 정보를 보고 의미하는 단어를 고르세요   
3     다음 정보를 보고 의미하는 단어를 고르세요   
4     다음 정보를 보고 의미하는 단어를 고르세요   
...                       ...   
1113  다음 정보를 보고 의미하는 단어를 고르세요   
1114  다음 정보를 보고 의미하는 단어를 고르세요   
1115  다음 정보를 보고 의미하는 단어를 고르세요   
1116  다음 정보를 보고 의미하는 단어를 고르세요   
1117  다음 정보를 보고 의미하는 단어를 고르세요   

                                                content answer  \
0                                 한곳에서 다른 곳으로 장소를 이동하다.     가다   
1      물건을 넣어 들거나 메고 다닐 수 있게 만든 용구. 가죽이나 천, 비닐 따위로 만든다.     가방   
2     옷감, 종이, 머리털 따위를 자르는 기구. 날이 있는 두 개의 쇠를 교차시켜 가운데...     가위   
3     손을 내밀어 그 모양에 따라 순서나 승부를 정하는 방법. 두 개의 손가락만 편 것을...  가위바위보   
4     한 해의 네 철 가운데 셋째 철. 여름과 겨울의 사이이며, 달로는 9~11월, 절기...     가을   
...                                                 ...    ...   
1113                                  혼인하여 남자의 짝이 된 여자.      처   
1114                                 목적을 이룰 때까지 뒤좇아 구함.     추구   
1115     